In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

engine = create_engine("sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3")
conlt = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

In [2]:
sql = '''
SELECT *
FROM dividends
LIMIT 1'''
struct = pd.read_sql(sql, conpf)
struct.dtypes

id            int64
stock_id      int64
name         object
year          int64
quarter       int64
number        int64
ppu         float64
amt         float64
net         float64
x_date       object
p_date       object
dtype: object

In [3]:
sql = '''
SELECT YEAR(p_date) AS p_year, MONTH(p_date) AS p_month, name, year, quarter, number, ppu, amt, net, x_date, p_date
FROM dividends
WHERE YEAR(p_date) = 2021
'''
dtl_by_month = pd.read_sql(sql, conpf)
dtl_by_month.shape

(106, 11)

In [4]:
ttl_by_month = dtl_by_month.groupby(['p_year','p_month','name'], as_index=True).agg(
    {
        'net':['sum','count'],
    }
)
ttl_by_month

net      
                            sum count
p_year p_month name                  
2021   1       ORI      5670.00     1
       2       LPF      1231.65     1
       3       CPTGF    1620.00     1
               DIF     20880.00     1
               EGATIF   2000.00     1
...                         ...   ...
       12      MCS     25200.00     1
               POPF     2223.00     1
               RCL      6075.00     1
               RJH      2970.00     1
               STA     16875.00     1

[105 rows x 2 columns]

In [5]:
ttl_by_month.sum()

net  sum      700874.5
     count       106.0
dtype: float64

In [6]:
ttl_by_month.groupby(level='p_month').sum()

net      
               sum count
p_month                 
1          5670.00     1
2          1231.65     1
3         71491.44     9
4        105808.20    12
5        121199.76    21
6         77048.75    10
7          1350.00     1
8          9180.00     3
9        191048.50    31
10         8550.00     2
11         1800.00     1
12       106496.20    14

In [7]:
sql = '''
SELECT name, p_date, FORMAT(amt,2) AS amt, FORMAT(net,2) AS net, format(amt-net,2) AS refund, year, quarter AS q
FROM dividends
WHERE YEAR(p_date) = 2021
ORDER BY p_date DESC'''
dtl_by_month = pd.read_sql(sql, conpf)
dtl_by_month

,name,p_date,amt,net,refund,year,q
0,HREIT,2021-12-23,"1,738.00","1,564.20",173.80,2021,3
1,EGATIF,2021-12-17,640.00,640.00,0.00,2021,3
2,EGATIF,2021-12-17,"1,040.00","1,040.00",0.00,2021,3
3,MCS,2021-12-09,"25,900.00","25,200.00",700.00,2021,3
4,IVL,2021-12-09,"5,250.00","4,725.00",525.00,2021,3
...,...,...,...,...,...,...,...
101,DIF,2021-03-12,"20,880.00","20,880.00",0.00,2020,4
102,MC,2021-03-11,"3,500.00","3,150.00",350.00,2020,2
103,GVREIT,2021-03-10,"3,666.60","3,299.94",366.66,2020,4
104,LPF,2021-02-15,"1,368.50","1,231.65",136.85,2020,3


In [8]:
sql = '''
SELECT YEAR(p_date) AS pay_year, SUM(amt) AS grs_amt, SUM(net) AS net_amt, SUM(amt-net) AS refund
FROM dividends
GROUP BY pay_year
ORDER BY pay_year DESC'''
ttl_by_year = pd.read_sql(sql, conpf)
ttl_by_year

,pay_year,grs_amt,net_amt,refund
0,2022,19601.00,19206.90,394.10
1,2021,752315.10,700874.50,51440.60
2,2020,616228.94,576030.84,40198.10
3,2019,552424.34,510266.46,42157.88
4,2018,351895.60,309725.58,42170.02
5,2017,245981.10,219514.23,26466.87


In [9]:
dtl_by_month[mask].count()

NameError: name 'mask' is not defined

In [ ]:
sql = '''
SELECT name, COUNT(*) AS qtrs, SUM(net) AS ttl_net
FROM dividends
GROUP BY name
ORDER BY SUM(net) DESC'''
total = pd.read_sql(sql, conpf)
total.head(30)

### Summarized profit by name

In [ ]:
sql = '''
SELECT name, sum(profit) AS profit
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
GROUP BY stocks.name
ORDER BY sum(profit) DESC LIMIT 10'''
profits_by_stock = pd.read_sql(sql, conpf)
profits_by_stock

In [ ]:
sql = '''
SELECT YEAR(date) AS year, MONTH(date) AS month, profit
FROM sells 
WHERE YEAR(date) = 2021
ORDER BY YEAR(date) DESC, MONTH(date) DESC
'''
sells = pd.read_sql(sql, conpf)
sells

In [ ]:
grouped = sells.groupby(['year','month'])
grouped

In [ ]:
profit_by_month = grouped['profit'].sum()
profit_by_month

In [ ]:
grouped.agg(['sum','mean', 'max', 'min','count'])

### Temporary process to calculate dividend portion of profit

In [ ]:
year = 2020
quarter = 4

In [ ]:
sql = """
SELECT name, year, quarter, aq_eps, ay_eps
FROM epss 
WHERE year = %s AND quarter = %s
"""
sql = sql % (year, quarter)
df_epss = pd.read_sql(sql, conlt)
df_epss.head()

In [ ]:
sql = '''
SELECT name, dividend
FROM dividend 
'''
df_dividend = pd.read_sql(sql, const)
df_dividend

In [ ]:
df_merge = pd.merge(df_epss, df_dividend, on='name', how='inner')
df_merge['xxx_pct'] = df_merge.dividend / df_merge.aq_eps * 100
df_merge.set_index('name',inplace=True)
df_merge.sort_values(['xxx_pct'],ascending=[True])

In [ ]:
df_merge.loc['MCS']